環境はRNA_sqe

In [ ]:
conda activate RNAseq

seqkitのインストールコマンド（一度インストールすれば次回から不要）

In [ ]:
mamba install -c bioconda seqkit

fastqファイルの中身を見る

In [ ]:
seqkit stats *_001.fastq.gz > statistics.tsv

seqkit statsコマンドでfastqファイルをタブで分ける/ statistics.tvsは出力されるファイル名（任意に変更可能）

In [ ]:
seqkit stats -T *_001.fastq.gz > statistics.tsv

出力されたデータをexcelで開き表を成型する。その後適切なファイル名に変更し保存する。

アダプター配列を検索して、それを抜いた配列を表示させる
"|"　コマンドを左から順番に実行させる
"-v"　検索でヒットした配列を抜く
"@"　


"wc" ヒットした行数を表示する
wc -l

In [ ]:
grep  -v "@" R1.fastq |grep "ATTCCATA" |wc -l

In [ ]:
grep  -v "@" R1.fastq |grep "ATTCCATA" |less

fastpを使うとリードの品質が低い領域のトリミングとアダプター除去を
１度に実行できる（ブログ）。Phred qualityスコア20以下のリード領域をトリミング”-q 20”
-t, -T 1は塩基の左右の一塩基を強制的に取り除いている（NGSだと最初と最後の一文字の精度が低いため）
-w 8 CUPを8個使うという事

In [ ]:
fastp -i WT-2_R1_001.fastq.gz -I WT-2_R2_001.fastq.gz -3 -o out_pair1.fq.gz -O out_pair2.fq.gz \-h report.html -j report.json -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1

出力されたreporファイル(HTMLファイル)を開く

より精度を高めother adapter sequenceを減らすためアダプター配列(pdfに書いてある)を指定して抜くコマンド

In [ ]:
fastp -i WT-2_R1_001.fastq.gz -I WT-2_R2_001.fastq.gz -3 -o out_pair1.fq.gz -O out_pair2.fq.gz \-h report.html -j report.json -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1 --adapter_sequence ATTCCATA --adapter_sequence_r2 CGCTCCAC  

同様にHTMLのレポートを開いてother adapter sequenceが減っていることを確認する
（今回は何故か増えた）

他の物でも同じように実行する

In [ ]:
fastp -i dg237_R1_001.fastq.gz -I dg237_R2_001.fastq.gz -3 -o out_pair1_dg237_R1.fq.gz -O out_pair2_dg237_R2.fq.gz \-h report_dg237.html -j report_dg237.json -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1 --adapter_sequence AGGTCAGA --adapter_sequence_r2 CGCTCCAC

In [ ]:
fastp -i BR10PR22_R1_001.fastq.gz -I BR10PR22_R2_001.fastq.gz -3 -o out_pair1_BR10PR22_R1.fq.gz -O out_pair2_BR10PR22_R2.fq.gz \-h report_BR10PR22.html -j report_BR10PR22.json -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1 --adapter_sequence GACGAGAT --adapter_sequence_r2 ATTAACAA

いったんここまででゲノムデータの掃除が終わり

ここからDNAのマッピングに入る
まずはインデックスを引く
インデックスをつける作業は1つのリファレンスゲノムに対して1回だけでOK

In [ ]:
bowtie2-build GCA_002142475.1.fna bowite2_index

インデックスファイルに対して、各サンプル（1サンプル当たり2つのファイル、先ほどきれいに整えたデータ、拡張子は.gzのままで大丈夫）をindexを用いてアラインメントさせる

In [ ]:
bowtie2 --sensitive -x bowite2_index -1 out_pair1.fq.gz -2 out_pair2.fq.gz |samtools sort -@ 8 - >sample1_WT-2.bam

出力されたbamファイルもインデックスをつけてあげる

In [ ]:
samtools index -@ 4 sample1_WT-2.bam

In [ ]:
最後にマッピング率を調べる。

In [ ]:
samtools flagstats sample1_WT-2.bam

上手くいったコマンド（トレーニングで）

In [ ]:
fastp_opts="-3 -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1"

# fastpコマンドの実行
fastp -i WT-2_R1_001.fastq.gz -I WT-2_R2_001.fastq.gz -o out_pair1_WT-2.fq.gz -O out_pair2_WT-2.fq.gz -h report_WT-2.html -j report_WT-2.json --adapter_sequence ATTCCATA --adapter_sequence_r2 CGCTCCAC $fastp_opts || { echo "WT-2でエラーが発生しました。"; exit 1; }

fastp -i dg237_R1_001.fastq.gz -I dg237_R2_001.fastq.gz -o out_pair1_dg237.fq.gz -O out_pair2_dg237.fq.gz -h report_dg237.html -j report_dg237.json --adapter_sequence AGGTCAGA --adapter_sequence_r2 CGCTCCAC $fastp_opts || { echo "dg237でエラーが発生しました。"; exit 1; }

# bowtie2のインデックス作成
bowtie2-build GCA_002142475.1.fna bowtie2_index || { echo "bowtie2-buildでエラーが発生しました。"; exit 1; }

# bowtie2によるマッピングとsamtoolsによるソート
bowtie2 --sensitive -x bowtie2_index -1 out_pair1_WT-2.fq.gz -2 out_pair2_WT-2.fq.gz | samtools sort -@ 8 -o WT-2.bam || { echo "WT-2のマッピングでエラーが発生しました。"; exit 1; }

bowtie2 --sensitive -x bowtie2_index -1 out_pair1_dg237.fq.gz -2 out_pair2_dg237.fq.gz | samtools sort -@ 8 -o dg237.bam || { echo "dg237のマッピングでエラーが発生しました。"; exit 1; }

# samtoolsによるインデックス作成
samtools index -@ 4 WT-2.bam || { echo "WT-2.bamのインデックス作成でエラーが発生しました。"; exit 1; }
samtools index -@ 4 dg237.bam || { echo "dg237.bamのインデックス作成でエラーが発生しました。"; exit 1; }

# samtools flagstatsの実行と結果のファイル出力
samtools flagstats WT-2.bam > WT-2_flagstats.txt || { echo "WT-2.bamのflagstatsでエラーが発生しました。"; exit 1; }
samtools flagstats dg237.bam > dg237_flagstats.txt || { echo "dg237.bamのflagstatsでエラーが発生しました。"; exit 1; }


完成品

In [ ]:
# Step 1: 初期FASTQ統計情報
seqkit stats -T *_001.fastq.gz > statistics.fastq.tsv || { echo "seqkit stats エラー"; exit 1; }

# Step 2: fastpによるトリミングとレポート生成
fastp_opts="-3 -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1"
for sample in "WT" "WT-2" "dg230" "dg230-2" "dg231" "dg231-2" "dg232" "dg232-2" "dg233" "dg233-2" "dg234" "dg234-2" "dg235" "dg235-2" "dg236" "dg236-2" "dg237" "dg237-2" "dg238" "dg238-2" "dg239" "dg239-2" "dg240" "dg240-2" "dg241" "dg241-2" "dg242" "dg242-2"; do
  case $sample in
    "WT") adapter1="GAACTGAG"; adapter2="CGCTCCAC";;
    "WT-2") adapter1="ATTCCATA"; adapter2="CGCTCCAC";;
    "dg230") adapter1="GAACTGAG"; adapter2="TATCTTGT";;
    "dg230-2") adapter1="ATTCCATA"; adapter2="TATCTTGT";;
    "dg231") adapter1="GAACTGAG"; adapter2="AGCTACTA";;
    "dg231-2") adapter1="ATTCCATA"; adapter2="CCACCAGG";;
    "dg232") adapter1="GAACTGAG"; adapter2="CCACCAGG";;
    "dg232-2") adapter1="ATTCCATA"; adapter2="CCACCAGG";;
    "dg233") adapter1="GAACTGAG"; adapter2="AGGATAAT";;
    "dg233-2") adapter1="ATTCCATA"; adapter2="AGGATAAT";;
    "dg234") adapter1="GAACTGAG"; adapter2="ACAAGTGG";;
    "dg234-2") adapter1="ATTCCATA"; adapter2="ACAAGTGG";;
    "dg235") adapter1="GAACTGAG"; adapter2="TACTGTTC";;
    "dg235-2") adapter1="ATTCCATA"; adapter2="TACTGTTC";;
    "dg236") adapter1="GAACTGAG"; adapter2="ATTAACAA";;
    "dg236-2") adapter1="ATTCCATA"; adapter2="ATTAACAA";;
    "dg237") adapter1="AGGTCAGA"; adapter2="CGCTCCAC";;
    "dg237-2") adapter1="GACGAGAT"; adapter2="CGCTCCAC";;
    "dg238") adapter1="AGGTCAGA"; adapter2="TATCTTGT";;
    "dg238-2") adapter1="GACGAGAT"; adapter2="TATCTTGT";;
    "dg239") adapter1="AGGTCAGA"; adapter2="AGCTACTA";;
    "dg239-2") adapter1="GACGAGAT"; adapter2="AGCTACTA";;
    "dg240") adapter1="AGGTCAGA"; adapter2="CCACCAGG";;
    "dg240-2") adapter1="GACGAGAT"; adapter2="CCACCAGG";;
    "dg241") adapter1="AGGTCAGA"; adapter2="AGGATAAT";;
    "dg241-2") adapter1="GACGAGAT"; adapter2="AGGATAAT";;
    "dg242") adapter1="AGGTCAGA"; adapter2="ACAAGTGG";;
    "dg242-2") adapter1="GACGAGAT"; adapter2="ACAAGTGG";;
  esac
  fastp -i ${sample}_R1_001.fastq.gz -I ${sample}_R2_001.fastq.gz \
        -o out_pair1_${sample}.fq.gz -O out_pair2_${sample}.fq.gz \
        -h report_${sample}.html -j report_${sample}.json \
        --adapter_sequence $adapter1 --adapter_sequence_r2 $adapter2 $fastp_opts || { echo "fastp エラー - $sample"; exit 1; }
done

# Step 3: トリミング後のFASTQ統計情報
seqkit stats -T out_pair*.fq.gz > statistics.fq.tsv || { echo "seqkit stats エラー"; exit 1; }

# Step 4: Bowtie2インデックス作成
bowtie2-build GCA_002142475.1.fna bowtie2_index || { echo "Bowtie2インデックス作成エラー"; exit 1; }

# Step 5: アライメントとソート
for sample in "WT" "WT-2" "dg230" "dg230-2" "dg231" "dg231-2"  "dg232" "dg232-2" "dg233" "dg233-2" "dg234" "dg234-2" "dg235" "dg235-2" "dg236" "dg236-2" "dg237" "dg237-2" "dg238" "dg238-2" "dg239" "dg239-2" "dg240" "dg240-2" "dg241" "dg241-2" "dg242" "dg242-2"; do
  bowtie2 --sensitive -x bowtie2_index -p 8 -1 out_pair1_${sample}.fq.gz -2 out_pair2_${sample}.fq.gz | \
  samtools sort -@ 8 -o ${sample}.bam || { echo "Bowtie2アライメントエラー - $sample"; exit 1; }
done

# Step 6: BAMインデックス作成
for sample in "WT" "WT-2" "dg230" "dg230-2" "dg231" "dg231-2" "dg232" "dg232-2" "dg233" "dg233-2" "dg234" "dg234-2" "dg235" "dg235-2" "dg236" "dg236-2" "dg237" "dg237-2" "dg238" "dg238-2" "dg239" "dg239-2" "dg240" "dg240-2" "dg241" "dg241-2" "dg242" "dg242-2"; do
  samtools index -@ 4 ${sample}.bam || { echo "samtoolsインデックス作成エラー - $sample"; exit 1; }
done

# Step 7: BAM統計情報
for sample in "WT" "WT-2" "dg230" "dg230-2" "dg231" "dg231-2" "dg232" "dg232-2" "dg233" "dg233-2" "dg234" "dg234-2" "dg235" "dg235-2" "dg236" "dg236-2" "dg237" "dg237-2" "dg238" "dg238-2" "dg239" "dg239-2" "dg240" "dg240-2" "dg241" "dg241-2" "dg242" "dg242-2"; do
  samtools flagstat ${sample}.bam > ${sample}_flagstats.txt || { echo "samtools flagstat エラー - $sample"; exit 1; }
done

# Step 8: BAM統計情報をまとめる
cat *_flagstats.txt > all_flagstats.txt || { echo "BAM統計情報の結合エラー"; exit 1; }


マッピング率が低かった２つのサンプルを改めて解析してみた。

In [ ]:
# Step 1: 初期FASTQ統計情報
seqkit stats -T *_001.fastq.gz > statistics.fastq.tsv || { echo "seqkit stats エラー"; exit 1; }

# Step 2: fastpによるトリミングとレポート生成
fastp_opts="-3 -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1"
for sample in "dg236-2" "dg238-2"; do
  case $sample in
    "dg236-2") adapter1="ATTCCATA"; adapter2="ATTAACAA";;
    "dg238-2") adapter1="GACGAGAT"; adapter2="TATCTTGT";;
  esac
  fastp -i ${sample}_R1_001.fastq.gz -I ${sample}_R2_001.fastq.gz \
        -o out_pair1_${sample}.fq.gz -O out_pair2_${sample}.fq.gz \
        -h report_${sample}.html -j report_${sample}.json \
        --adapter_sequence $adapter1 --adapter_sequence_r2 $adapter2 $fastp_opts || { echo "fastp エラー - $sample"; exit 1; }
done

# Step 3: トリミング後のFASTQ統計情報
seqkit stats -T out_pair*.fq.gz > statistics.fq.tsv || { echo "seqkit stats エラー"; exit 1; }

# Step 4: Bowtie2インデックス作成
bowtie2-build GCA_002142475.1.fna bowtie2_index || { echo "Bowtie2インデックス作成エラー"; exit 1; }

# Step 5: アライメントとソート
for sample in "dg236-2" "dg238-2"; do
  bowtie2 --sensitive -x bowtie2_index -p 8 -1 out_pair1_${sample}.fq.gz -2 out_pair2_${sample}.fq.gz | \
  samtools sort -@ 8 -o ${sample}.bam || { echo "Bowtie2アライメントエラー - $sample"; exit 1; }
done

# Step 6: BAMインデックス作成
for sample in "dg236-2" "dg238-2"; do
  samtools index -@ 4 ${sample}.bam || { echo "samtoolsインデックス作成エラー - $sample"; exit 1; }
done

# Step 7: BAM統計情報
for sample in "dg236-2" "dg238-2"; do
  samtools flagstat ${sample}.bam > ${sample}_flagstats.txt || { echo "samtools flagstat エラー - $sample"; exit 1; }
done

# Step 8: BAM統計情報をまとめる
cat *_flagstats.txt > all_flagstats.txt || { echo "BAM統計情報の結合エラー"; exit 1; }


IGVでアラインメントを可視化する

In [ ]:
igv -g GCA_002142475.1.fna sample1_WT-2.bam

breseqで変異が生じている場所を特定する。（IGVと合わせてほんとの変異かどうか確認する）※時間かかるので注意
WT-2_breseq_resultは出力フォルダ名で任意に

In [ ]:
 breseq -r Leptolyngbya_boryana_IAM101.gbk -o WT-2_breseq_relut WT-2_R1_001.fastq.gz WT-2_R2_001.fastq.gz

gdファイルを統合する

In [ ]:
gdtools COMPARE -o integrate.html -r Leptolyngbya_boryana_IAM101.gbk BR10PR22_output.gd dg237_output.gd WT-2_output.gd